In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
import wandb
wandb.login()

wandb: Currently logged in as: sahaana (use `wandb login --relogin` to force relogin)


True

# Imports 

In [4]:
import pandas as pd
import numpy as np
from collections import defaultdict

import matplotlib.pyplot as plt
from transformers import AutoTokenizer, DistilBertModel

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

In [5]:
import sys
#sys.path.append('/lfs/1/sahaana/enrichment/enrich/utils')
sys.path.append('/lfs/1/sahaana/enrichment/ember/utils')
 
    
from embedding_datasets import DeepMatcherDataset, EmberEvalDataset
from embedding_models import TripletSingleBERTModel
from embedding_utils import param_header, tokenize_batch  
from embedding_runner import train_model, eval_model
#from model_utils import MatchedDatasetTriplets, param_header_bert, tokenize_batch   
#from models import BatchedTripletSingleTowerModel, BatchedTripletSingleBERTModel
#from model_runner import train_model, eval_model
from knn_utils import FaissKNeighbors, knn_top_1_PRFS #, knn_matching_accuracy, find_perfect_recall

In [6]:
%load_ext autoreload
%autoreload 2

10


# BUMBUM

In [8]:
datasets = {0:"abt_buy_exp_data", 
            1:"amazon_google_exp_data", 
            2:"beer_exp_data", 
            3:"company_exp_data", 
            4:"dblp_acm_exp_data", 
            5:"dblp_scholar_exp_data", 
            6:"dirty_dblp_acm_exp_data", 
            7:"dirty_dblp_scholar_exp_data", 
            8:"dirty_itunes_amazon_exp_data", 
            9:"dirty_walmart_amazon_exp_data", 
            10:"fodors_zagat_exp_data", 
            11:"itunes_amazon_exp_data", 
            12:"walmart_amazon_exp_data"}

In [59]:
for d in range(10,11):
    print(datasets[d])
    left = f'/lfs/1/sahaana/enrichment/data/deepmatcher/{datasets[d]}/tableA_processed.pkl'
    right = f'/lfs/1/sahaana/enrichment/data/deepmatcher/{datasets[d]}/tableB_processed.pkl'

    left = pd.read_pickle(left)
    right = pd.read_pickle(right)

    train_df = f'/lfs/1/sahaana/enrichment/data/deepmatcher/{datasets[d]}/train_updated.csv'
    train_df = pd.read_csv(train_df)

    val_df = f'/lfs/1/sahaana/enrichment/data/deepmatcher/{datasets[d]}/val_updated.csv'
    val_df = pd.read_csv(val_df)

    test_df = f'/lfs/1/sahaana/enrichment/data/deepmatcher/{datasets[d]}/test_updated.csv'
    test_df = pd.read_csv(test_df)

    epochs = 5
    batch_size = 8
    final_size = 200
    lr = .00001
    tl_margin = 1.0
    tl_p = 2
    pool_type = "CLS"
    column = "merged_all"
    shuffle = True
    tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
    tokenizer_max_length = 512
    compute_val = False
    train_size = 50#len(train_df)

    bert_path=f'/lfs/1/sahaana/enrichment/ember/pretraining/models/{datasets[d]}-uncased-masked-ALL-BM25'
    bert_model = DistilBertModel.from_pretrained(bert_path, return_dict=True)

    model_name = f'{datasets[d]}-uncased-masked-ALL-BM25'

    train_data = DataLoader(DeepMatcherDataset(left, right, train_size, column, train_df), 
                            batch_size=batch_size,
                            shuffle = shuffle
                            )

    val_data = DataLoader(DeepMatcherDataset(left, right, len(val_df), column, val_df), 
                           batch_size=batch_size,
                           shuffle = False
                          )

    triplet_loss = nn.TripletMarginLoss(margin=tl_margin, p=tl_p)
    losses = []
    val_losses = []
    model = TripletSingleBERTModel(final_size, pool_type, bert_path)
    optimizer = optim.AdamW(model.parameters(), lr=lr)#optim.SGD(model.parameters(), lr=lr)

    save_dir = param_header(batch_size, final_size, lr, pool_type, epochs, train_size)
    save_dir = f'models/{model_name}/{save_dir}/'

    #wandb.init(project=model_name)

    train_model(model, 
                tokenizer, 
                tokenize_batch, 
                train_data, 
                val_data, 
                triplet_loss, 
                optimizer, 
                epochs, 
                losses, 
                val_losses, 
                save_dir, 
                compute_val, 
                tokenizer_max_length = tokenizer_max_length)


    left_data = DataLoader(EmberEvalDataset(left, column), 
                           batch_size=batch_size,
                           shuffle = False
                          )
    right_data = DataLoader(EmberEvalDataset(right, column), 
                           batch_size=batch_size,
                           shuffle = False
                          )

    left_embeddings = eval_model(model, tokenizer, left_data, tokenizer_max_length=512)
    right_embeddings = eval_model(model, tokenizer, right_data, tokenizer_max_length=512)

    knn = FaissKNeighbors(k=30)
    knn.fit(right_embeddings)
    neib = knn.kneighbors(left_embeddings)
    print(datasets[d])
    print(f"precision, recall, F1, support, predicted: {knn_top_1_PRFS(neib[0], neib[1], test_df)}")
    print()
    print()
    print()

fodors_zagat_exp_data
fodors_zagat_exp_data
precision, recall, F1, support, predicted: ((0.8, 0.7272727272727273, 0.761904761904762, None), [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])





In [56]:
left

,id,name,addr,city,phone,type,class,merged_all
0,0,` arnie morton \ 's of chicago ',' 435 s. la cienega blv . ',` los angeles ',310/246 -1501,american,0,[SEP] name ` arnie morton \ 's of chicago ' ...
1,1,` art \ 's delicatessen ',' 12224 ventura blvd. ',` studio city ',818/762 -1221,american,1,[SEP] name ` art \ 's delicatessen ' [SEP] ...
2,2,` hotel bel-air ',' 701 stone canyon rd. ',` bel air ',310/472 -1211,californian,2,[SEP] name ` hotel bel-air ' [SEP] addr ' 7...
3,3,` cafe bizou ',' 14016 ventura blvd. ',` sherman oaks ',818/788 -3536,french,3,[SEP] name ` cafe bizou ' [SEP] addr ' 1401...
4,4,campanile,' 624 s. la brea ave. ',` los angeles ',213/938 -1447,american,4,[SEP] name campanile [SEP] addr ' 624 s. la...
...,...,...,...,...,...,...,...,...
528,528,` yank sing ',' 427 battery st. ',` san francisco ',415/541 -4949,asian,528,[SEP] name ` yank sing ' [SEP] addr ' 427 b...
529,529,` yaya cuisine ',' 1220 9th ave. ',` san francisco ',415/566 -6966,` greek and middle eastern ',529,[SEP] name ` yaya cuisine ' [SEP] addr ' 12...
530,530,` yoyo tsumami bistro ',' 1611 post st. ',` san francisco ',415/922 -7788,french,530,[SEP] name ` yoyo tsumami bistro ' [SEP] ad...
531,531,zarzuela,' 2000 hyde st. ',` san francisco ',415/346 -0800,` mexican/latin american/spanish ',531,[SEP] name zarzuela [SEP] addr ' 2000 hyde ...


In [57]:
right

,id,name,addr,city,phone,type,class,merged_all
0,0,` apple pan the ',' 10801 w. pico blvd. ',` west la ',310-475-3585,american,534,[SEP] name ` apple pan the ' [SEP] addr ' 1...
1,1,` asahi ramen ',' 2027 sawtelle blvd. ',` west la ',310-479-2231,` noodle shops ',535,[SEP] name ` asahi ramen ' [SEP] addr ' 202...
2,2,` baja fresh ',' 3345 kimber dr. ',` westlake village ',805-498-4049,mexican,536,[SEP] name ` baja fresh ' [SEP] addr ' 3345...
3,3,` belvedere the ',' 9882 little santa monica blvd. ',` beverly hills ',310-788-2306,` pacific new wave ',537,[SEP] name ` belvedere the ' [SEP] addr ' 9...
4,4,` benita \ 's frites ',' 1433 third st. promenade ',` santa monica ',310-458-2889,` fast food ',538,[SEP] name ` benita \ 's frites ' [SEP] add...
...,...,...,...,...,...,...,...,...
326,326,` plumpjack cafe ',' 3127 fillmore st. ',` san francisco ',415-563-4755,` american ( new ) ',108,[SEP] name ` plumpjack cafe ' [SEP] addr ' ...
327,327,postrio,' 545 post st. ',` san francisco ',415-776-7825,californian,109,[SEP] name postrio [SEP] addr ' 545 post st...
328,328,` ritz-carlton dining room ( san francisco ) ',' 600 stockton st. ',` san francisco ',415-296-7465,` french ( new ) ',110,[SEP] name ` ritz-carlton dining room ( san ...
329,329,` rose pistola ',' 532 columbus ave. ',` san francisco ',415-399-0499,italian,111,[SEP] name ` rose pistola ' [SEP] addr ' 53...


In [17]:
test_df

,ltable_id,rtable_id,label
0,6,230,0
1,22,240,1
2,501,214,0
3,38,256,1
4,70,130,0
...,...,...,...
184,108,326,1
185,133,229,0
186,86,156,0
187,424,125,0


In [20]:
left.loc[108]

id                                                          108
name                                         ` plumpjack cafe '
addr                                      ' 3201 fillmore st. '
city                                          ` san francisco '
phone                                             415/563 -4755
type                                              mediterranean
class                                                       108
merged_all     [SEP]  name ` plumpjack cafe ' [SEP]  addr ' ...
Name: 108, dtype: object

In [21]:
right.loc[326]

id                                                          326
name                                         ` plumpjack cafe '
addr                                      ' 3127 fillmore st. '
city                                          ` san francisco '
phone                                              415-563-4755
type                                       ` american ( new ) '
class                                                       108
merged_all     [SEP]  name ` plumpjack cafe ' [SEP]  addr ' ...
Name: 326, dtype: object

In [26]:
neib[1][108]

array([326, 323, 187,  57,  32, 214,  40, 327,   0, 221,  26,  42, 320,
       177, 200, 322,  11,  55,  62, 235, 192, 191,   4,   8, 318,  33,
       317, 314, 198, 227])

In [30]:
(_,_,_,_),predicted = knn_top_1_PRFS(neib[0], neib[1], test_df)

In [31]:
predicted

[0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0]

In [37]:
test_df[test_df['label'].to_numpy() != np.array(predicted)]

,ltable_id,rtable_id,label
5,384,300,0
10,197,251,0
15,292,89,0
18,131,233,0
64,75,293,1
136,428,137,0


In [39]:
left.loc[384]

id                                                          384
name                               ` beesley \ 's of buckhead '
addr                                ' 260 e. paces ferry road '
city                                                    atlanta
phone                                             404/264 -1334
type                                                continental
class                                                       384
merged_all     [SEP]  name ` beesley \ 's of buckhead ' [SEP...
Name: 384, dtype: object

In [40]:
right.loc[300]

id                                                          300
name                              ` hedgerose heights inn the '
addr                              ' 490 e. paces ferry rd. ne '
city                                                    atlanta
phone                                              404-233-7673
type                                                continental
class                                                        82
merged_all     [SEP]  name ` hedgerose heights inn the ' [SE...
Name: 300, dtype: object

In [41]:
neib[1][384]

array([300, 141, 296, 166, 298, 180, 294, 142, 147, 155, 148, 178, 159,
       150, 146, 175, 153, 145, 157, 143, 305, 306, 152, 151, 307, 299,
       172, 183, 184, 170])

In [49]:
neib[1][75]

array([172, 298, 300, 178, 157, 175, 147, 145, 180, 141, 299, 142, 149,
       150, 294, 148, 330, 154, 309, 306, 177, 166, 307, 297, 176, 296,
       153, 143, 155, 308])

In [50]:
left.loc[75]

id                                                           75
name                                                bacchanalia
addr                 ' 3125 piedmont rd. . near peachtree rd. '
city                                                    atlanta
phone                                             404/365 -0410
type                                              international
class                                                        75
merged_all     [SEP]  name bacchanalia [SEP]  addr ' 3125 pi...
Name: 75, dtype: object

In [53]:
right.loc[293]

id                                                          293
name                                                bacchanalia
addr                                      ' 3125 piedmont rd. '
city                                                    atlanta
phone                                              404-365-0410
type                                                californian
class                                                        75
merged_all     [SEP]  name bacchanalia [SEP]  addr ' 3125 pi...
Name: 293, dtype: object

In [54]:
right.loc[172]

id                                                          172
name                                                       nava
addr                                     ' 3060 peachtree rd. '
city                                                    atlanta
phone                                              404-240-1984
type                                               southwestern
class                                                       706
merged_all     [SEP]  name nava [SEP]  addr ' 3060 peachtree...
Name: 172, dtype: object